<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DL0120EN-SkillsNetwork/images/IDSN-logo.png" width="300" alt="cognitiveclass.ai logo">


# Lab: Scaling on CPU and GPU

Welcome to the lab, first we wanna make sure we are on TensorFlow 2.x

Please start with 1 vCPU and then use 2, 4, 8 and 16 (and optionally 1 or 2 GPUs)

Note down execution times and compute/plot scale up factor


In [1]:
!pip install tensorflow==2.9.0

**Restart kernel for latest version of TensorFlow to be activated, then continue to the next section.**


In [2]:
import tensorflow as tf
import time
import numpy as np
print(tf.__version__)



from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if not tf.__version__ == '2.9.0':
    printmd('<<<<<!!!!! Please install TensorFlow 2.9.0 and PLEASE RESTART the jupyter kernel using the menue above !!!!!>>>>>')



2024-03-05 18:29:29.620157: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-05 18:29:29.626482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-05 18:29:29.626508: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.0


Let's check out if there are GPUs available


In [3]:
if len(tf.config.experimental.list_physical_devices('GPU')) > 0:
    print('GPUs found, this will be fun!')

2024-03-05 18:29:39.633009: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-05 18:29:39.633067: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-05 18:29:39.633102: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterlab-kalpeshpati1): /proc/driver/nvidia/version does not exist


Let's create some data (e.g. two random matrices of shape 10000x10000)


In [4]:
size = 10000
a = tf.random.uniform(shape=[size,size])
b = tf.random.uniform(shape=[size,size])

2024-03-05 18:29:49.941817: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Let's multiply them together and measure time (execute the following cell at least 3 times and take the minimum to account for cache misses). Please run this notebook in different runtimes configurations to get the execution times.


To change the execution environment configuration please click on the ![image.png](attachment:image.png) symbol within Watson Studio. In the "Environment" tab you can choose and change the "Environment definition".


We recommend that you go the the "Environment" tab in the Watson Studio Project settings and create the environments first ![image.png](attachment:image.png)


Once done, it should look similar to this
![image.png](attachment:image.png)


In [5]:
start = time.time()
c = tf.matmul(a,b)
print(time.time()-start)

29.35025191307068


Please update the execution times below for the different runtime configurations with your values (1,2,4,8,16 CPU, optional 1,2,4 GPU)


In [6]:
execution_times_cpu = {
    'cpu_1x' : 35.3118622303009,
    'cpu_2x' : 15.228885173797607,
    'cpu_4x' : 10.392901182174683,
    'cpu_8x' : 6.591029644012451,
    'cpu_16x' : 2.9781200885772705  
}
execution_times_cpu

{'cpu_1x': 35.3118622303009,
 'cpu_2x': 15.228885173797607,
 'cpu_4x': 10.392901182174683,
 'cpu_8x': 6.591029644012451,
 'cpu_16x': 2.9781200885772705}

In [ ]:
execution_times_gpu = {
    'gpu_1x' : 0.0012249946594238281,
    'gpu_2x' : 0.0007076263427734375,
    'gpu_4x' : 0.0006804466247558594   
}
execution_times_gpu

Merge CPU and GPU results


In [ ]:
execution_times_cpu_gpu = {}
execution_times_cpu_gpu.update(execution_times_cpu)
execution_times_cpu_gpu.update(execution_times_gpu)
execution_times_cpu_gpu

Compute normalized speedup factors for CPU, GPU and combindes runs


In [ ]:
max_time_cpu = np.max(list(execution_times_cpu.values()))
max_time_gpu = np.max(list(execution_times_gpu.values()))
max_time_cpu_gpu = np.max(list(execution_times_cpu_gpu.values()))

In [ ]:
execution_times_norm_cpu = np.array(1)/(np.array(list(execution_times_cpu.values()))/max_time_cpu)
execution_times_norm_gpu = np.array(1)/(np.array(list(execution_times_gpu.values()))/max_time_gpu)
execution_times_norm_cpu_gpu = np.array(1)/(np.array(list(execution_times_cpu_gpu.values()))/max_time_cpu_gpu)

Plot the results


In [ ]:
%matplotlib inline
import seaborn as sns
sns.barplot(x=np.array(list(execution_times_cpu.keys())), y=execution_times_norm_cpu).set(ylabel='speedup')

In [ ]:
sns.barplot(x=np.array(list(execution_times_gpu.keys())), y=execution_times_norm_gpu).set(ylabel='speedup')

In [ ]:
sns.barplot(x=np.array(list(execution_times_cpu_gpu.keys())), y=execution_times_norm_cpu_gpu).set(ylabel='speedup')

As you can clearly see going from a single CPU core to a single GPU gives us a speedup of more than four orders of magnitude. So large scale deep learning network training has to be done on GPUs if possible
